# DS340 Final Project - Using ML to generate music
Ben Gardiner

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
# import sklearn
!pip install pretty_midi
import pretty_midi
from tqdm import tqdm
import random 
import re
import collections 
import pathlib 
import glob 
warnings.simplefilter("ignore", UserWarning)

# Introduction

For this assignment I wanted to generate unique artifical music using machine learning. Music can be represented a sequence of sounds following each other, so I collected data in time series format and used a LSTM model to predict the sound that would come next.

# Methodology

1. get data 
2. preprocess data 
3. build modeel
4. train it 
5. generate new music

# Getting data 

The Maestro dataset is a collection of over 200 hours of classical piano performances in MIDI and audio formats, spanning a wide range of musical styles and composers. It includes annotations such as tempo, dynamics, and pedal usage, and is commonly used for machine learning, music information retrieval, and computer music research. It can be downloaded for non-commercial use from the Magenta project website.

https://magenta.tensorflow.org/datasets/maestro

In [ ]:
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

midi_paths = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(midi_paths))

In [ ]:
sample_file = midi_paths[0]
sample_object = pretty_midi.PrettyMIDI(sample_file)

# Preprocessing our data

Right now our data is a bunch of MIDI files stored in our directory, but we need to transform these files into something the neural network can consume. We will preprocess the data with the following steps...

1. take a MIDI object and turn it into a sequence of notes 
2. visualize the song by plotting a sequence of notes 
3. iterate through all the songs in the dataset and collect song data (sequence of notes, name of song, length of second (seconds), etc.)

In [ ]:

def midi_to_notes(midi_object):
  # source - https://www.tensorflow.org/tutorials/audio/music_generation
  instrument = midi_object.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start
  
  for note in sorted_notes:
    start = note.start
    end = note.end
    pitch = note.pitch
    notes['pitch'].append(pitch)
    notes['note_name'].append(pretty_midi.note_number_to_name(pitch)) # e.g 50 -> D3
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

midi_to_notes(sample_object)


In [ ]:
def plot_piano_roll(notes: pd.DataFrame, count: int = None):
  """Used to visualize the notes in a track."""
  # source - https://www.tensorflow.org/tutorials/audio/music_generation

  if count:
    title = f'First {count} notes'
  else:
    title = f'Whole track'
    count = len(notes['pitch'])
  plt.figure(figsize=(20, 4))
  plot_pitch = np.stack([notes['pitch'], notes['pitch']], axis=0)
  plot_start_stop = np.stack([notes['start'], notes['end']], axis=0)
  plt.plot(
      plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
  plt.xlabel('Time [s]')
  plt.ylabel('Pitch')
  _ = plt.title(title)

plot_piano_roll(midi_to_notes(sample_object),count=10)
plot_piano_roll(midi_to_notes(sample_object))

In [ ]:
def get_song_data(file_path):
    """Returns song name, length, number of notes, and vectorized notes for an INDIVIDUAL song"""
    midi_object = pretty_midi.PrettyMIDI(file_path)

    # length of song in seconds
    song_length = midi_object.get_end_time()

    # name of song
    song_name = os.path.basename(file_path)

    # notes df 
    notes_df = midi_to_notes(midi_object)
    num_notes = len(notes_df)

    # sequences of notes in array form e.g [50, 51, 52, 53, 54]
    vectorized_notes = notes_df[['pitch','step','duration']].values

    return song_name, song_length, num_notes, vectorized_notes
    
get_song_data(sample_file)

In [7]:
def get_data(midi_paths):
    data = []
    for path in tqdm(midi_paths):
        try:
            song_name, song_length, num_notes, vectorized_notes = get_song_data(path)
            song_data = [song_name, song_length, num_notes, vectorized_notes]
            
            # # we only want songs that are less than 5 minutes
            # if song_length < 300: 
            #     data.append(song_data)
            data.append(song_data)

            # else:
            #     continue
        except:
            print(f"Error with {path}")

    df = pd.DataFrame(data, columns=["song name", "song length (seconds)", "number of notes", "vectorized_notes"])
    # print(df.head())

    all_notes = df['vectorized_notes']

    return all_notes,df

percent = 1
num_files = int(len(midi_paths)*percent)
random_paths = random.sample(midi_paths,num_files)
all_notes, df = get_data(random_paths)
# all_notes, df = get_data(midi_paths[:5])


df.head()

  1%|          | 7/1282 [00:05<13:10,  1.61it/s]Exception ignored in: <function _xla_gc_callback at 0x7f97d5ad7820>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
  1%|          | 14/1282 [00:06<07:42,  2.74it/s]

Error with data/maestro-v2.0.0/2009/MIDI-Unprocessed_13_R1_2009_04_ORIG_MID--AUDIO_13_R1_2009_13_R1_2009_04_WAV.midi


100%|██████████| 1282/1282 [07:40<00:00,  2.78it/s]


,song name,song length (seconds),number of notes,vectorized_notes
0,MIDI-Unprocessed_Recital20_MID--AUDIO_20_R1_20...,2098.052083,23835,"[[73.0, 0.0, 0.38020833333333326], [64.0, 0.05..."
1,MIDI-Unprocessed_13_R1_2008_01-04_ORIG_MID--AU...,667.652344,7886,"[[81.0, 0.0, 0.08333333333333326], [76.0, 0.04..."
2,MIDI-Unprocessed_R2_D2-12-13-15_mid--AUDIO-fro...,647.133333,8775,"[[46.0, 0.0, 0.10208333333333353], [34.0, 0.01..."
3,MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_201...,1091.661458,8038,"[[50.0, 0.0, 1.0625], [74.0, 0.006510416666666..."
4,MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AU...,768.143750,6699,"[[39.0, 0.0, 0.10312499999999991], [51.0, 0.00..."


### Creating input and output sequences

Right now our data is a list of songs where each song is a sequence of notes. However, we need to reshape our data into something the model can understand, so we need to create input and output sequences.

We iterate through all the data and collect 64 note sequences and the note that follows. We'll train the model on the 64 note sequences and hope it learns what note should follow. Just like you could use a time series model to predict where a stock might be based on where it was in the past.

In [8]:
from tensorflow.keras.utils import to_categorical

def create_input_output(data, sequence_length=25):
    input_seqs = []
    output_notes = []

    for song in data:
        for i in range(0, len(song) - sequence_length):
            input_seq = song[i:i+sequence_length]
            output_note = song[i+sequence_length]

            input_seqs.append(input_seq)
            output_notes.append(output_note)

    input_seqs = np.array(input_seqs)
    output_seqs = np.array([
        # to_categorical(np.array([note[0] for note in output_notes])), # pitch
        np.array([note[0] for note in output_notes]), # pitch
        np.array([note[1] for note in output_notes]), # step
        np.array([note[2] for note in output_notes]) # duration
    ]).transpose()

    return input_seqs, output_seqs

input_seqs, output_seqs = create_input_output(all_notes)

print("Input Sequences")
print(type(input_seqs))
print(input_seqs.shape)

print("Output Sequences")
print(type(output_seqs))
print(output_seqs.shape)

del all_notes,df 

Input Sequences
<class 'numpy.ndarray'>
(7075850, 25, 3)
Output Sequences
<class 'numpy.ndarray'>
(7075850, 3)


### Creating a train and test set

In [9]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(input_seqs, output_seqs, test_size=0.2)

del input_seqs, output_seqs


# Designing the model

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Sequential
from keras.layers import Lambda
import keras.backend as K


In [11]:
# create a LSTM model using input_seqs and output_seqs

input_shape = (25, 3)
learning_rate = 0.005

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)

# 3 elements of the note output
pitch_probs = tf.keras.layers.Dense(128, activation="softmax", name='pitch_probs')(x) # returns softmax "prob" for each pitch number
step = tf.keras.layers.Dense(1, activation="linear",name='step')(x)
duration = tf.keras.layers.Dense(1, activation="linear",name='duration')(x)

# combining three elements 
# outputs = {'pitch': pitch, 'step': step, 'duration': duration}
outputs = {'pitch': pitch_probs, 'step': step, 'duration': duration}


model = tf.keras.Model(inputs, outputs)

loss = {'pitch': "sparse_categorical_crossentropy", 'step': "mse", 'duration': "mse"}


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss=loss,
    loss_weights={
        'pitch': 0.05,
        'step': 1.0,
        'duration':1.0,
    },
    optimizer=optimizer,
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 25, 3)]      0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 128)          67584       ['input_1[0][0]']                
                                                                                                  
 duration (Dense)               (None, 1)            129         ['lstm[0][0]']                   
                                                                                                  
 pitch_probs (Dense)            (None, 128)          16512       ['lstm[0][0]']                   
                                                                                              

# Training the model


In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

epochs = 100

# create labels for each output
pitch_labels = y_train[:, 0].astype(int)
step_labels = y_train[:, 1].reshape(-1, 1)
duration_labels = y_train[:, 2].reshape(-1, 1)

# batch size
batch_size = 16

# train the model
history = model.fit(
    x=x_train,
    y={"pitch": pitch_labels, "step": step_labels, "duration": duration_labels}, 
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
)

# save model after running
model.save('model.h5')



Epoch 1/100
33722/88449 [==========>...................] - ETA: 5:36 - loss: 0.3483 - duration_loss: 0.1014 - pitch_probs_loss: 3.8588 - step_loss: 0.0539

Analyzing its accuracy over time

# Generate New Music!

In [ ]:
def generate_next_note(input_sequence, model, temperature):
    # Make sure the input sequence has the correct shape
    if len(input_sequence) < 25:
        input_sequence = np.pad(input_sequence, ((25-len(input_sequence), 0), (0, 0)), 'constant')

    # Convert the input sequence to a format suitable for the model
    input_sequence = np.array([input_sequence])

    # Generate the output probabilities using the model
    output_probs = model.predict(input_sequence)

    # Modify the pitch probabilities using the temperature parameter
    pitch_probs = output_probs["pitch"][0] # Extract the pitch probabilities from the output
    pitch_probs = np.log(pitch_probs) / temperature
    pitch_probs = np.exp(pitch_probs) / np.sum(np.exp(pitch_probs))

    # Sample the next pitch from the modified probabilities
    pitch_index = np.random.choice(len(pitch_probs), p=pitch_probs)
    next_pitch = pitch_index

    # Modify the step and duration predictions using the temperature parameter
    step_mean = output_probs["step"][0][0]
    step_stddev = np.exp(output_probs["step"][0][0] / temperature)
    next_step = np.random.normal(loc=step_mean, scale=step_stddev)

    duration_mean = output_probs["duration"][0][0]
    duration_stddev = np.exp(output_probs["duration"][0][0] / temperature)
    next_duration = np.random.normal(loc=duration_mean, scale=duration_stddev)

    return [next_pitch, next_step, next_duration]


In [ ]:
sample_input = x_test[0]

print("Sample input:")
print(sample_input[:5])
print("Sample input shape",sample_input.shape)

In [ ]:
new_note = generate_next_note(sample_input,model,5)
print(new_note)

In [ ]:
def generate_music(model,input,length,temperature):
  new_song = []
  prev_start = 0
  for i in range(length):
    # generating a new note with the model
    next_note = generate_next_note(sample_input,model,temperature)

    # details of new note
    pitch = next_note[0]
    step = next_note[1]
    duration = next_note[2]
    
    start = prev_start + step
    end = start + duration
    prev_start = start
  
    new_song.append([pitch,step,duration,start,end])

  # plot_piano_roll(new_song)

  song_df = pd.DataFrame(
      data=new_song,
      columns=["pitch","step","duration","start","end"]
  )

  return song_df

In [ ]:
song = generate_music(model,sample_input,1000,1)
print("Input")
print(song[:25])
print("New Music")
print(song[25:])

In [ ]:
song

In [ ]:
plot_piano_roll(song)

In [ ]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument = pretty_midi.Instrument(program=0),
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  # source - https://www.tensorflow.org/tutorials/audio/music_generation

  pm = pretty_midi.PrettyMIDI()
  # instrument = pretty_midi.Instrument(
  #     program=pretty_midi.instrument_name_to_program(
  #         instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [ ]:
pm = notes_to_midi(song,"song.mid")
pm